In [1]:
import pandas as pd
import sys
import os
import numpy as np
import xarray as xr
import zarr
import glob
from rechunker import rechunk


import dask
from ipywidgets import interact
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
from dask import delayed, compute
import dask.array as da
import warnings
from re import split
import matplotlib.pyplot as plt
import time
warnings.filterwarnings('ignore')
sys.path.insert(1, f"{os.path.abspath(os.path.join(os.path.abspath(''), '../'))}")
from src.utils import time_3d, get_pars_from_ini
location = split(', |_|-|!', os.popen('hostname').read())[0].replace("\n", "")
path_data = get_pars_from_ini(campaign='loc')[location]['path_data']
plt.rcParams['animation.html'] = 'jshtml'
%matplotlib inline


In [10]:
cluster = SLURMCluster(queue="seseml",
                       memory='150GB',
                       cores=20,
                       processes=1,
                       walltime='23:40:00',
                       scheduler_options={'host': '172.22.179.3:7222', 'dashboard_address': ':7330'})


In [11]:
# cluster.adapt(maximum_jobs=4)
cluster.scale(4)
cluster

In [12]:
client = Client(cluster)
client

Client Scheduler: tcp://172.22.179.3:7222 Dashboard: http://172.22.179.3:7330/status,Cluster Workers: 4 Cores: 80 Memory: 558.80 GiB


In [13]:
def rechunk_wrap(path, max_mem = '10GB'):
    target_store = path.replace('zarr/','zarr_rckd/')
    ds = xr.open_zarr(path)
    a = ds.dims
    del ds
    source_group = zarr.open(path)
    temp_store = '/data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/temp'
    target_chunks = {}
    for i in source_group.array_keys():
        dims = {}
        dim = source_group[i].attrs.asdict()['_ARRAY_DIMENSIONS']
        for k in dim:
            if k == 'time':
                if a[k] < 2000:
                    dims[k] = a[k]
                else:
                    dims[k] = 2000
            elif k == 'range':
                dims[k] = a[k]
            elif k == 'cross_track':
                dims[k] = a[k]
            elif k == 'bin_size':
                dims[k] = a[k]
            elif k == 'vector':
                dims[k] = a[k]
            elif k == 'params':
                dims[k] = a[k]
        target_chunks[i] = dims

    try:    
        rechunked = rechunk(source=source_group, 
                            target_chunks=target_chunks, 
                            max_mem=max_mem, 
                            target_store=target_store, 
                            temp_store=temp_store, 
                            executor='dask')
        rechunked.execute()
        zarr.convenience.consolidate_metadata(target_store)
    except ValueError as e:
        print(path, e)

In [14]:
folders = [i for i in glob.glob(f'{path_data}/zarr/*/*') if i.endswith('.zarr')]
len(folders)

42

In [ ]:
%%time
for j, fold in enumerate(folders):
    print(j)
    rechunk_wrap(fold)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [9]:
client.close()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
